In [1]:
import os
import pandas as pd
import yaml

In [2]:
with open("../../enriched_experiments.yml", "r") as f:
    exps = yaml.safe_load(f)

In [3]:
experiments = exps["experiments"]

In [23]:
experiments[-1].keys()

dict_keys(['slurm_id', 'experiment_nm', 'wandb_urls', 'val/vasr_vit/loss', 'train/vasr_vit/loss_epoch', 'val/loss', 'train/vasr_vit/accuracy_vasr_epoch', 'val/vasr_vit/accuracy_vasr', 'max_epoch', 'loss/best_ckpt', 'datasets'])

In [24]:
experiments

[{'slurm_id': [832505],
  'experiment_nm': 'bongard_hoi_vasr_images',
  'test_nm': 'lr_check',
  'wandb_urls': ['https://wandb.ai/avr-fighters/AVR_universal/runs/2a485jne'],
  'val/vasr_images/loss': {'best_loss': 0.024717137217521667,
   'best_loss_epoch': 2,
   'best_loss_step': 46049,
   'best_slurm_id': 832505},
  'train/bongard_hoi_images/loss_epoch': {'best_loss': 0.0,
   'best_loss_epoch': 0,
   'best_loss_step': 15349,
   'best_slurm_id': 832505},
  'train/vasr_images/loss_epoch': {'best_loss': 0.007650455459952354,
   'best_loss_epoch': 2,
   'best_loss_step': 46049,
   'best_slurm_id': 832505},
  'val/loss': {'best_loss': 0.02481253817677498,
   'best_loss_epoch': 2,
   'best_loss_step': 46049,
   'best_slurm_id': 832505},
  'val/bongard_hoi_images/loss': {'best_loss': 0.025924356654286385,
   'best_loss_epoch': 2,
   'best_loss_step': 46049,
   'best_slurm_id': 832505},
  'max_epoch': 2,
  'datasets': ['vasr_images', 'bongard_hoi_images'],
  'additional_inforamations': {'lr'

In [25]:
list(filter(lambda x: "accuracy" in x, experiments[-1].keys()))

['train/vasr_vit/accuracy_vasr_epoch', 'val/vasr_vit/accuracy_vasr']

In [26]:
# Get all runs that produced some accuracy results
unprocessed_count = 0
processed_count = 0
structured_results = []
for experiment in experiments:
    accuracy_cols = [key for key in experiment.keys() if "accuracy" in key]
    if len(accuracy_cols) == 0:
        # TODO: later collect STSN runs
        unprocessed_count += 1
        continue
    structured_results.append(
        {col: experiment[col]["best_loss"] for col in accuracy_cols}
        | {
            "loss/best_ckpt": experiment.get("loss/best_ckpt"),
            "wandb_urls": ",".join(experiment.get("wandb_urls")),
            "experiment_nm": experiment.get("experiment_nm"),
            "test_nm": experiment.get("test_nm"),
            "based_on/slurm_id": experiment.get("based_on", {}).get("slurm_id"),
            "max_epoch": experiment.get("max_epoch"),
        }
    )
    # TODO: enrich with information from "additional_inforamations" (TODO: add relevant fields in enrich_experiments.py)
    processed_count += 1
print(f"Processed {processed_count} experiments")
print(f"Unprocessed {unprocessed_count} experiments")

Processed 208 experiments
Unprocessed 34 experiments


In [7]:
# TODO: split by dataset (not mutually exclusive -- lets duplicate the information)
# VAEC/VASR/HOI/LOGO/LOGO+HOI/VAEC+VASR/LOGO+VAEC/HOI+VASR

# might be nice to add identifiers different than slurm_ids
# (autoincrement value would work but could be hard to
# automatically assign it so that it makes sense in paper)

In [27]:
df = pd.DataFrame(structured_results)

In [28]:
df

,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy_epoch,loss/best_ckpt,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,train/vasr/accuracy_epoch,val/vasr/accuracy,...,test/bongard_logo_test_ff_vit_2/accuracy_bongard_logo,test/bongard_logo_test_hd_novel_vit_2/accuracy_bongard_logo,test/bongard_hoi_unseen-unseen_vit_2/accuracy_bongard_logo,test/bongard_hoi_seen-seen_vit_2/accuracy_bongard_logo,test/bongard_hoi_unseen-seen_vit_2/accuracy_bongard_logo,test/vaec_test2_vit_2/accuracy_vasr,test/vaec_test5_vit_2/accuracy_vasr,test/vaec_test1_vit_2/accuracy_vasr,test/vaec_test3_vit_2/accuracy_vasr,test/vaec_test4_vit_2/accuracy_vasr
0,0.604056,0.778655,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring,frozen_slot,839928.0,43,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.580541,0.832082,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,42,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,0.809366,0.252280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.607584,0.834469,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,0.803173,0.286448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,combined_vasr_vaec_fine_tune_no_asymetric,NaN,282,NaN,NaN,...,0.66,0.659375,0.709057,0.644549,0.644673,NaN,NaN,NaN,NaN,NaN
204,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v2,combined_vasr_vaec_fine_tune_no_asymetric,NaN,52,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.309801,0.292105,0.296061,0.312903,0.278003
205,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_combined_v2,combined_vasr_vaec_fine_tune_no_asymetric,NaN,214,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,NaN,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_baseline_v2,None,NaN,33,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# normalize columns

In [9]:
cols = df.columns

In [10]:
cols

Index(['val/bongard_hoi/accuracy', 'train/bongard_hoi/accuracy_epoch',
       'loss/best_ckpt', 'wandb_urls', 'experiment_nm', 'test_nm',
       'based_on/slurm_id', 'max_epoch', 'train/vasr/accuracy_epoch',
       'val/vasr/accuracy',
       ...
       'test/bongard_logo_test_ff_vit_2/accuracy_bongard_logo',
       'test/bongard_logo_test_hd_novel_vit_2/accuracy_bongard_logo',
       'test/bongard_hoi_unseen-unseen_vit_2/accuracy_bongard_logo',
       'test/bongard_hoi_seen-seen_vit_2/accuracy_bongard_logo',
       'test/bongard_hoi_unseen-seen_vit_2/accuracy_bongard_logo',
       'test/vaec_test2_vit_2/accuracy_vasr',
       'test/vaec_test5_vit_2/accuracy_vasr',
       'test/vaec_test1_vit_2/accuracy_vasr',
       'test/vaec_test3_vit_2/accuracy_vasr',
       'test/vaec_test4_vit_2/accuracy_vasr'],
      dtype='object', length=106)

In [11]:
train_acc = [col for col in cols if "train" in col and "accuracy" in col]
val_acc = [col for col in cols if "val" in col and "accuracy" in col]

In [12]:
datasets = ["bongard_logo", "bongard_hoi", "vaec", "vasr"]

In [13]:
for dataset in datasets:
    _cols = [col for col in train_acc if dataset in col]
    df[f"train/{dataset}/accuracy"] = df[_cols].bfill(axis=1).iloc[:, 0]

    _cols = [col for col in val_acc if dataset in col]
    df[f"val/{dataset}/accuracy"] = df[_cols].bfill(axis=1).iloc[:, 0]

In [14]:
df = df.drop(
    columns=set(train_acc) - {f"train/{dataset}/accuracy" for dataset in datasets}
)
df = df.drop(columns=set(val_acc) - {f"val/{dataset}/accuracy" for dataset in datasets})

In [15]:
test_acc = [col for col in cols if "test" in col and "accuracy" in col]

In [16]:
test_types = [
    ("bongard_logo", "test_bd"),
    ("bongard_logo", "test_ff"),
    ("bongard_logo", "test_hd_comb"),
    ("bongard_logo", "test_hd_novel"),
    ("vaec", "test1"),
    ("vaec", "test2"),
    ("vaec", "test3"),
    ("vaec", "test4"),
    ("vaec", "test5"),
    ("bongard_hoi", "seen-seen"),
    ("bongard_hoi", "seen-unseen"),
    ("bongard_hoi", "unseen-seen"),
    ("bongard_hoi", "unseen-unseen"),
    ("vasr", "vasr"),
]

In [17]:
for dataset, test_type in test_types:
    _cols = [col for col in test_acc if test_type in col]
    df[f"test/{dataset}/{test_type}"] = df[_cols].bfill(axis=1).iloc[:, 0]

In [18]:
df = df.drop(columns=test_acc)

In [19]:
df.columns

Index(['val/bongard_hoi/accuracy', 'loss/best_ckpt', 'wandb_urls',
       'experiment_nm', 'test_nm', 'based_on/slurm_id', 'max_epoch',
       'val/vasr/accuracy', 'val/bongard_logo/accuracy', 'val/vaec/accuracy',
       'train/bongard_logo/accuracy', 'train/bongard_hoi/accuracy',
       'train/vaec/accuracy', 'train/vasr/accuracy',
       'test/bongard_logo/test_bd', 'test/bongard_logo/test_ff',
       'test/bongard_logo/test_hd_comb', 'test/bongard_logo/test_hd_novel',
       'test/vaec/test1', 'test/vaec/test2', 'test/vaec/test3',
       'test/vaec/test4', 'test/vaec/test5', 'test/bongard_hoi/seen-seen',
       'test/bongard_hoi/seen-unseen', 'test/bongard_hoi/unseen-seen',
       'test/bongard_hoi/unseen-unseen', 'test/vasr/vasr'],
      dtype='object')

In [20]:
df.head()

,val/bongard_hoi/accuracy,loss/best_ckpt,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/vasr/accuracy,val/bongard_logo/accuracy,val/vaec/accuracy,...,test/vaec/test1,test/vaec/test2,test/vaec/test3,test/vaec/test4,test/vaec/test5,test/bongard_hoi/seen-seen,test/bongard_hoi/seen-unseen,test/bongard_hoi/unseen-seen,test/bongard_hoi/unseen-unseen,test/vasr/vasr
0,0.604056,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring,frozen_slot,839928.0,43,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.580541,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,42,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,0.252280,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.607584,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_scoring_full_train,trained_slot_aux_test,839928.0,34,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,/mnt/evafs/groups/mandziuk-lab/akaminski/model...,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_scoring_full_train,trained_slot_aux_test,839928.0,5,0.286448,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
for dataset, test_type in test_types:
    _dataset_cols = [col for col in df.columns if dataset in col and "test" not in col]
    cols = [
        "wandb_urls",
        "experiment_nm",
        "test_nm",
        "based_on/slurm_id",
        "max_epoch",
        *_dataset_cols,
    ]
    display(
        df[[f"test/{dataset}/{test_type}"] + cols]
        .sort_values(f"test/{dataset}/{test_type}", ascending=False)
        .head(5)
    )

,test/bongard_logo/test_bd,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
25,0.881250,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871045.0,413,0.807778,0.822796
54,0.864583,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871045.0,174,0.773333,0.764194
41,0.833333,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871039.0,172,0.748889,0.766237
115,0.833333,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,combined_MLP_scoring_none_activation,NaN,240,0.793333,0.981613
122,0.833333,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v2,transformer_scoring_none_activation,NaN,240,0.793333,0.981613


,test/bongard_logo/test_ff,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
25,0.765000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871045.0,413,0.807778,0.822796
41,0.755000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871039.0,172,0.748889,0.766237
54,0.725000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871045.0,174,0.773333,0.764194
53,0.721667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871045.0,255,0.755556,0.764839
42,0.711667,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871039.0,137,0.751111,0.753441


,test/bongard_logo/test_hd_comb,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
25,0.710,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871045.0,413,0.807778,0.822796
122,0.705,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_transformer_v2,transformer_scoring_none_activation,NaN,240,0.793333,0.981613
115,0.705,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,combined_MLP_scoring_none_activation,NaN,240,0.793333,0.981613
191,0.695,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,combined_bongard_hoi_bongard_logo_fine_tune,NaN,215,0.808889,0.984301
54,0.695,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871045.0,174,0.773333,0.764194


,test/bongard_logo/test_hd_novel,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_logo/accuracy,train/bongard_logo/accuracy
191,0.731250,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,combined_bongard_hoi_bongard_logo_fine_tune,NaN,215,0.808889,0.984301
140,0.725000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_combined_v2,combined_parameter_search_no_context_norm,NaN,213,0.786667,0.938387
182,0.715625,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v3,combined_separate_scoring,NaN,153,0.797778,0.984761
42,0.700000,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,trained_slot_aux_test,871039.0,137,0.751111,0.753441
25,0.693750,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_logo_scoring,frozen_slot,871045.0,413,0.807778,0.822796


,test/vaec/test1,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/vaec/accuracy,train/vaec/accuracy
61,0.996722,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,trained_slot_aux_test,871052.0,114,0.997057,1.0
33,0.994131,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871037.0,73,0.993889,1.0
37,0.988054,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,train_slot_no_aux,871052.0,115,0.988237,1.0
46,0.986088,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,trained_slot_aux_test,871039.0,208,0.987817,1.0
62,0.976190,https://wandb.ai/avr_universal/AVR_universal/r...,vaec_scoring,trained_slot_aux_test,871052.0,115,0.983356,1.0


...

,test/bongard_hoi/seen-seen,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy
178,0.757373,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_logo_frozen_relational_no_asymetrical...,NaN,277,0.830688,0.809817
174,0.754915,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_logo_frozen_relational_no_asymetrical,NaN,291,0.827454,0.807213
135,0.753575,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_parameter_search_no_asymetrical_no_co...,NaN,223,0.825397,0.824660
188,0.751206,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v3,combined_separate_scoring_no_asymetric,NaN,208,0.828924,0.917278
131,0.751117,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_parameter_search_no_asymetrical,NaN,365,0.830982,0.934899


,test/bongard_hoi/seen-unseen,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy
186,0.855363,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v3,combined_separate_scoring_no_asymetric,NaN,172,0.832745,0.922833
188,0.852021,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v3,combined_separate_scoring_no_asymetric,NaN,208,0.828924,0.917278
174,0.848678,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_logo_frozen_relational_no_asymetrical,NaN,291,0.827454,0.807213
131,0.848071,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_parameter_search_no_asymetrical,NaN,365,0.830982,0.934899
135,0.846247,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_parameter_search_no_asymetrical_no_co...,NaN,223,0.825397,0.824660


,test/bongard_hoi/unseen-seen,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy
188,0.751206,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v3,combined_separate_scoring_no_asymetric,NaN,208,0.828924,0.917278
178,0.750548,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_logo_frozen_relational_no_asymetrical...,NaN,277,0.830688,0.809817
194,0.746383,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_bongard_hoi_bongard_logo_fine_tune_no...,NaN,279,0.825985,0.948744
174,0.746164,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_logo_frozen_relational_no_asymetrical,NaN,291,0.827454,0.807213
186,0.745068,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v3,combined_separate_scoring_no_asymetric,NaN,172,0.832745,0.922833


,test/bongard_hoi/unseen-unseen,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/bongard_hoi/accuracy,train/bongard_hoi/accuracy
188,0.819479,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vaec_combined_v3,combined_separate_scoring_no_asymetric,NaN,208,0.828924,0.917278
131,0.819479,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_combined_v2,combined_parameter_search_no_asymetrical,NaN,365,0.830982,0.934899
186,0.816377,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v3,combined_separate_scoring_no_asymetric,NaN,172,0.832745,0.922833
117,0.815136,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v2,combined_MLP_scoring_none_activation_no_asymet...,NaN,184,0.810112,0.937503
149,0.815136,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_bongard_logo_combined_v2,combined_comparing_same_relationals,NaN,172,0.806584,0.998785


,test/vasr/vasr,wandb_urls,experiment_nm,test_nm,based_on/slurm_id,max_epoch,val/vasr/accuracy,train/vasr/accuracy
200,0.523393,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v2,combined_vasr_vaec_fine_tune,NaN,46,0.522059,0.982372
124,0.518581,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_transformer_v2,transformer_scoring_none_activation,NaN,46,0.506833,0.991863
118,0.518581,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v2,combined_MLP_scoring_none_activation,NaN,58,0.506833,0.991863
192,0.511408,https://wandb.ai/avr_universal/AVR_universal/r...,vasr_combined_v2,combined_bongard_hoi_bongard_logo_fine_tune,NaN,44,0.511704,0.967699
183,0.478539,https://wandb.ai/avr_universal/AVR_universal/r...,bongard_hoi_vasr_combined_v3,combined_separate_scoring,NaN,28,0.492194,0.987019


In [22]:
df.to_csv("../../structured_results.csv", index=False)

In [ ]:
# split by dataset 